# Notebook to play with plotting financial charts

Latest version: 2024-08-18  
Author: MvS

## Description


## Result




In [2]:
import yfinance
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go

periods = [200, 50]

dt_end = datetime.datetime.today()
dt_data_start = dt_end - datetime.timedelta(days=max(periods) * 3)

stock = "FROG"

# Grab the stock data
load_df = yfinance.download(
    stock, start=dt_data_start.strftime('%Y-%m-%d'), end=dt_end.strftime('%Y-%m-%d')
)

[*********************100%%**********************]  1 of 1 completed


In [34]:
stock_df = load_df.copy()

# Compute the simple moving average (SMA)
for period in periods:
    stock_df[f"SMA_{period:03}"] = stock_df["Close"].rolling(window=period).mean()

# Now that we calculated the SMA, we can remove the data points before the actual AOI that we want.
stock_df = stock_df[-max(periods) :].copy()


dt_start = stock_df.index[0]

print(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {stock_df.shape} data points."""
)

#stock_df[[f"SMA_{min(periods):03}", f"SMA_{max(periods):03}", 'Close']].plot()
fig = go.Figure(data=[
          go.Candlestick(
                x=stock_df.index,
                open=stock_df['Open'],
                high=stock_df['High'],
                low=stock_df['Low'],
                close=stock_df['Close'],
                name = f"{stock}",            
          ),
          go.Scatter(
                x=stock_df.index,
                y=stock_df[f"SMA_{max(periods):03}"],
                mode='lines',
                name = f"SMA_{max(periods):03}",
                line = dict(color='#FF5252', width=2, dash='dash'),
          ),
          go.Scatter(
                x=stock_df.index,
                y=stock_df[f"SMA_{min(periods):03}"],
                mode='lines',
                name = f"SMA_{min(periods):03}",
                line = dict(color='#311B92', width=2, dash='dash'),
          ),
])

fig.update_yaxes(type='log')

fig.update_layout(
    title='Standard candlesticks and SMAs',
    yaxis_title=f"{stock} Stock",
    shapes = [dict(
        x0='2023-12-09', x1='2023-12-09', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2023-12-09', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='Some event')],
    xaxis_rangeslider_visible=False,
    width=1200,
    height=800,
)

fig.show()


Calculating SMAs for [200, 50] length starting from:
  2023-10-31 and ending on 2024-08-18
  and containing (200, 8) data points.


### Color defaults

[Trading view](https://www.tradingview.com/pine-script-docs/concepts/colors/#constant-colors)


| Name          | Hex Code | RGB                     | Sample                                                |
|---------------|----------|-------------------------|-------------------------------------------------------|
| color.aqua    | #00BCD4  | color.rgb(0, 188, 212)  | <span style="color:#00BCD4;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.black   | #363A45  | color.rgb(54, 58, 69)   | <span style="color:#363A45;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.blue    | #2196F3  | color.rgb(33, 150, 243) | <span style="color:#2196F3;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.fuchsia | #E040FB  | color.rgb(224, 64, 251) | <span style="color:#E040FB;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.gray    | #787B86  | color.rgb(120, 123, 134)| <span style="color:#787B86;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.green   | #4CAF50  | color.rgb(76, 175, 80)  | <span style="color:#4CAF50;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.lime    | #00E676  | color.rgb(0, 230, 118)  | <span style="color:#00E676;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.maroon  | #880E4F  | color.rgb(136, 14, 79)  | <span style="color:#880E4F;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.navy    | #311B92  | color.rgb(49, 27, 146)  | <span style="color:#311B92;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.olive   | #808000  | color.rgb(128, 128, 0)  | <span style="color:#808000;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.orange  | #FF9800  | color.rgb(255, 152, 0)  | <span style="color:#FF9800;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.purple  | #9C27B0  | color.rgb(156, 39, 176) | <span style="color:#9C27B0;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.red     | #FF5252  | color.rgb(255, 82, 82)  | <span style="color:#FF5252;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.silver  | #B2B5BE  | color.rgb(178, 181, 190)| <span style="color:#B2B5BE;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.teal    | #00897B  | color.rgb(0, 137, 123)  | <span style="color:#00897B;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.white   | #FFFFFF  | color.rgb(255, 255, 255)| <span style="color:#FFFFFF;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
| color.yellow  | #FFEB3B  | color.rgb(255, 235, 59) | <span style="color:#FFEB3B;">&#9608;&#9608;&#9608;&#9608;&#9608;&#9608;</span> |
